In [ ]:
!pip install gensim

In [ ]:
# importing the dependencies
import numpy as np
import pandas as pd
import gensim
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
df = pd.read_csv('/content/IMDB Dataset.csv', engine='python', on_bad_lines='skip')

In [ ]:
story = []
for doc in df['review']:
  raw_data = sent_tokenize(doc)
  for data in raw_data:
    story.append(simple_preprocess(data))

In [ ]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2
)

In [ ]:
# building the vocab
model.build_vocab(story)

None


In [ ]:
# training
model.train(story,total_examples=model.corpus_count,epochs=model.epochs)

(29554651, 38938590)

In [ ]:
y = model.wv.index_to_key

In [ ]:
len(y)

53927

In [ ]:
# creating vector for review
from gensim.utils import simple_preprocess

def review_vector(doc_text):
  # Preprocess the document text to match the vocabulary's format
  processed_words = simple_preprocess(doc_text)

  # Filter words that are present in the vocabulary 'y'
  valid_words = [word for word in processed_words if word in y]

  if valid_words:
    # If there are valid words, get their vectors and compute the mean
    return np.mean(model.wv[valid_words], axis=0)
  else:
    # If no valid words are found, return a zero vector
    # model.wv.vector_size provides the dimension of the word vectors
    return np.zeros(model.wv.vector_size)

In [ ]:
review_vector(df['review'].values[0])

array([-0.28659683,  0.88581485, -0.08205818, -0.42676416,  0.38880762,
       -0.6727667 , -0.25117776, -0.1144669 ,  0.2651066 ,  0.01673012,
       -0.08697987, -0.2825848 , -0.17028204,  0.1477451 ,  0.51579607,
        0.67909986,  0.35835484,  0.26040483,  0.16170439,  0.09806266,
        0.71607333,  0.06991914, -0.11449487,  0.13343006, -0.19123071,
        0.7869586 , -0.13060325,  0.31789944, -0.29927224, -0.34662664,
        0.11818077,  0.24210607,  0.46426824, -0.40049437,  0.04259898,
       -0.4837816 ,  0.29123306, -0.2873268 , -0.19112907, -0.14346123,
        0.21546656,  0.0904894 ,  0.21774176,  0.05686488,  0.40206844,
        0.42473644,  0.21390648, -1.0006633 , -0.11376081,  0.56735754,
        0.50518274,  0.20489798,  0.05028726, -0.13582946, -0.42081952,
       -0.27628195,  0.09958383,  0.29870653, -0.05231849, -0.33009142,
       -0.47023845, -0.17177308,  0.06023604,  0.5568253 , -0.41920486,
       -0.71939826,  0.09909087, -0.10049833,  0.6291242 ,  0.49

In [ ]:
!pip install tdqm

In [ ]:
from tqdm import tqdm
x = []
for doc in tqdm(df['review'].values):
  x.append(review_vector(doc))

100%|██████████| 34831/34831 [01:14<00:00, 467.81it/s]


In [ ]:
x = np.array(x)

In [ ]:
x.shape

(34831, 100)

In [ ]:
# label encoding
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y = encoder.fit_transform(df['sentiment'])

In [ ]:
# train test split
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=1)

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score,confusion_matrix

In [ ]:
# using GaussianNB algo
gnb = GaussianNB()
gnb.fit(x_train,y_train)
y_pred = gnb.predict(x_test)
accuracy_score(y_test,y_pred)

0.7238409645471509

In [ ]:
# using randomforest
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(x_train,y_train)
y_pred2 = rf.predict(x_test)
accuracy_score(y_test,y_pred2)

0.8078082388402469

In [ ]:
# by using GaussianNB algo we get accuracy of 72%
# and by using randomforest model we get accuracy of 80%